In [4]:
# Metodo simplex

EPS = 1e-12

# Utilidades de lectura
def leer_int(msg, min_val=None, max_val=None):
    valor = None
    listo = False
    while not listo:
        txt = input(msg).strip()
        ok = True
        try:
            valor_tmp = int(txt)
        except Exception:
            ok = False
        if ok:
            if min_val is not None:
                if valor_tmp < min_val:
                    ok = False
            if max_val is not None and ok:
                if valor_tmp > max_val:
                    ok = False
        if ok:
            valor = valor_tmp
            listo = True
        else:
            print("Entrada inválida. Intenta de nuevo.")
    return valor

def leer_float(msg):
    valor = None
    listo = False
    while not listo:
        txt = input(msg).strip()
        ok = True
        try:
            valor_tmp = float(txt)
        except Exception:
            ok = False
        if ok:
            valor = valor_tmp
            listo = True
        else:
            print("Entrada inválida. Intenta de nuevo.")
    return valor


# Construcción de la tabla
def construir_tabla(c, A, b):
    n = len(c)
    m = len(A)
    col_RHS = n + m

    filaZ = [0.0] * (col_RHS + 1)
    i = 0
    while i < (col_RHS + 1):
        filaZ[i] = 0.0
        i += 1
    filaZ[0] = 1.0
    j = 0
    while j < n:
        filaZ[1 + j] = -float(c[j])
        j += 1
    # slacks en Z ya están en 0; RHS = 0

    tabla = [filaZ]
    i = 0
    while i < m:
        fila = [0.0] * (col_RHS + 1)
        j = 0
        while j < n:
            fila[1 + j] = float(A[i][j])
            j += 1
        # identidad
        fila[1 + n + i] = 1.0
        fila[col_RHS] = float(b[i])
        tabla.append(fila)
        i += 1

    return tabla, n, m, col_RHS

# Elección de columna entrante
def elegir_columna_entrante(tabla, n, m, col_RHS):
    filaZ = tabla[0]
    j = 1
    col_elegida = -1
    mas_neg = 0.0
    while j < col_RHS:
        val = filaZ[j]
        if val < mas_neg - EPS or (abs(val - mas_neg) <= EPS and val < mas_neg):
            # más negativa
            mas_neg = val
            col_elegida = j
        j += 1
    return col_elegida  # -1 => óptimo

# Elección de fila saliente
def elegir_fila_saliente(tabla, col, m, col_RHS):
    i = 1  # filas de restricciones
    fila_elegida = -1
    mejor_ratio = None
    while i <= m:
        aij = tabla[i][col]
        if aij > EPS:
            rhs = tabla[i][col_RHS]
            ratio = rhs / aij
            if mejor_ratio is None:
                mejor_ratio = ratio
                fila_elegida = i
            else:
                if ratio < mejor_ratio - EPS or abs(ratio - mejor_ratio) <= EPS and i < fila_elegida:
                    mejor_ratio = ratio
                    fila_elegida = i
        i += 1
    return fila_elegida  # -1 => no acotado

# Pivotear
def pivotear(tabla, fila_pivote, col_pivote, col_RHS):
    pivote = tabla[fila_pivote][col_pivote]
    # Normalizar fila pivote
    j = 0
    ncols = col_RHS + 1
    while j < ncols:
        tabla[fila_pivote][j] = tabla[fila_pivote][j] / pivote
        j += 1

    # Anular columna pivote en otras filas
    i = 0
    nfilas = len(tabla)
    while i < nfilas:
        if i != fila_pivote:
            factor = tabla[i][col_pivote]
            if abs(factor) > EPS:
                j = 0
                while j < ncols:
                    tabla[i][j] = tabla[i][j] - factor * tabla[fila_pivote][j]
                    j += 1
        i += 1

# Impresión de tabla
def _fmt_num(x, prec):
    s = ("{0:." + str(prec) + "f}").format(x)
    return s

def imprimir_tabla(tabla, prec=4):
    m = len(tabla) - 1
    # Encabezados de fila
    print("Tabla actual:")
    i = 0
    while i < len(tabla):
        if i == 0:
            encabezado = "Z : "
        else:
            if i < 10:
                encabezado = "F0" + str(i) + ": "
            else:
                encabezado = "F" + str(i) + ": "
        fila_txt = encabezado
        j = 0
        while j < len(tabla[i]):
            fila_txt += _fmt_num(tabla[i][j], prec)
            if j < len(tabla[i]) - 1:
                fila_txt += "  "
            j += 1
        print(fila_txt)
        i += 1
    print("")

# Reconstrucción de solución básica
def _es_col_canonica(tabla, col, fila_obj, EPS):
    i = 0
    nfilas = len(tabla)
    uno_en = -1
    ok = True
    while i < nfilas:
        val = tabla[i][col]
        if i == fila_obj:
            if abs(val) > EPS:
                ok = False
        else:
            if abs(val - 1.0) <= EPS and uno_en == -1:
                uno_en = i
            elif abs(val) > EPS:
                ok = False
        i += 1
    return ok and (uno_en != -1), uno_en

def reconstruir_solucion(tabla, n, m, col_RHS):
    # x
    x = [0.0] * n
    j = 0
    while j < n:
        es_canon, fila = _es_col_canonica(tabla, 1 + j, 0, EPS)
        if es_canon and fila >= 1:
            x[j] = tabla[fila][col_RHS]
        else:
            x[j] = 0.0
        j += 1
    # slacks
    sl = [0.0] * m
    j = 0
    while j < m:
        es_canon, fila = _es_col_canonica(tabla, 1 + n + j, 0, EPS)
        if es_canon and fila >= 1:
            sl[j] = tabla[fila][col_RHS]
        else:
            sl[j] = 0.0
        j += 1
    Z = tabla[0][col_RHS]
    return Z, x, sl


# Simplex funcion principal
def simplex(c, A, b, mostrar_pasos=True, prec=4, max_iter=10000):
    tabla, n, m, col_RHS = construir_tabla(c, A, b)
    iteracion = 0
    estado = "EN_PROCESO"
    continuar = True

    while continuar:
        col = elegir_columna_entrante(tabla, n, m, col_RHS)
        if col == -1:
            estado = "OPTIMO"
            continuar = False
        else:
            fila = elegir_fila_saliente(tabla, col, m, col_RHS)
            if fila == -1:
                estado = "NO_ACOTADO"
                continuar = False
            else:
                if mostrar_pasos:
                    print("--> Pivote: fila {}, columna {} (valor {})".format(fila, col, _fmt_num(tabla[fila][col], prec)))
                pivotear(tabla, fila, col, col_RHS)
                if mostrar_pasos:
                    imprimir_tabla(tabla, prec=prec)
        iteracion = iteracion + 1
        if iteracion >= max_iter and continuar:
            # límite de iteraciones alcanzado
            continuar = False

    Z, x, sl = reconstruir_solucion(tabla, n, m, col_RHS)
    return {"estado": estado, "Z": Z, "x": x, "slacks": sl, "tabla": tabla}

# Entrada de modelo por consola
def input_modelo():
    print("=== Modelo: Max c^T x  s.a.  Ax <= b, x >= 0, b >= 0 ===")
    n = leer_int("Número de variables (n): ", 1, None)
    m = leer_int("Número de restricciones (m): ", 1, None)

    c = []
    idx = 0
    while idx < n:
        val = leer_float("c{}: ".format(idx + 1))
        c.append(val)
        idx += 1

    A = []
    b = []
    i = 0
    while i < m:
        print("Restricción R{}:".format(i + 1))
        fila = []
        j = 0
        while j < n:
            val = leer_float("  a{}{}: ".format(i + 1, j + 1))
            fila.append(val)
            j += 1
        A.append(fila)
        bi = leer_float("  b{} : ".format(i + 1))
        b.append(bi)
        i += 1

    return c, A, b

# Demo Kyndryl
def demo_kyndryl():
    # Max 60 A + 50 B
    c = [60.0, 50.0]
    # 3A + 2B <= 240
    # 2A + 1B <= 200
    # 1A + 2B <= 100
    A = [
        [3.0, 2.0],
        [2.0, 1.0],
        [1.0, 2.0]
    ]
    b = [240.0, 200.0, 100.0]
    return c, A, b

# Menú principal
def main():
    salir = False
    while not salir:
        print("===============================================")
        print(" Simplex por consola (estilo TORA)")
        print(" 1) Ingresar modelo y resolver")
        print(" 2) Demo Kyndryl (Max 60A + 50B)")
        print(" 3) Salir")
        print("===============================================")
        op = leer_int("Opción: ", 1, 3)

        if op == 1:
            c, A, b = input_modelo()
            resp_pasos = input("¿Mostrar pasos (s/n)? ").strip().lower()
            mostrar_pasos = (resp_pasos == "s")
            prec = leer_int("Decimales a imprimir (0-10): ", 0, 10)
            res = simplex(c, A, b, mostrar_pasos=mostrar_pasos, prec=prec)
            print("Estado:", res["estado"])
            print("Z*   :", _fmt_num(res["Z"], prec))
            # x
            i = 0
            salida_x = "x*   : ["
            while i < len(res["x"]):
                salida_x += _fmt_num(res["x"][i], prec)
                if i < len(res["x"]) - 1:
                    salida_x += ", "
                i += 1
            salida_x += "]"
            print(salida_x)
            # slacks
            i = 0
            salida_s = "Slacks: ["
            while i < len(res["slacks"]):
                salida_s += _fmt_num(res["slacks"][i], prec)
                if i < len(res["slacks"]) - 1:
                    salida_s += ", "
                i += 1
            salida_s += "]"
            print(salida_s)
            print("")
        elif op == 2:
            c, A, b = demo_kyndryl()
            resp_pasos = input("¿Mostrar pasos (s/n)? ").strip().lower()
            mostrar_pasos = (resp_pasos == "s")
            prec = leer_int("Decimales a imprimir (0-10): ", 0, 10)
            res = simplex(c, A, b, mostrar_pasos=mostrar_pasos, prec=prec)
            print("Estado:", res["estado"])
            print("Z*   :", _fmt_num(res["Z"], prec))
            i = 0
            salida_x = "x*   : ["
            while i < len(res["x"]):
                salida_x += _fmt_num(res["x"][i], prec)
                if i < len(res["x"]) - 1:
                    salida_x += ", "
                i += 1
            salida_x += "]"
            print(salida_x)
            i = 0
            salida_s = "Slacks: ["
            while i < len(res["slacks"]):
                salida_s += _fmt_num(res["slacks"][i], prec)
                if i < len(res["slacks"]) - 1:
                    salida_s += ", "
                i += 1
            salida_s += "]"
            print(salida_s)
            print("")
        else:
            salir = True
# main()

In [3]:
main()

 Simplex por consola (estilo TORA)
 1) Ingresar modelo y resolver
 2) Demo Kyndryl (Max 60A + 50B)
 3) Salir
Opción: 2
¿Mostrar pasos (s/n)? s
Decimales a imprimir (0-10): 4
--> Pivote: fila 1, columna 1 (valor 3.0000)
Tabla actual:
Z : 1.0000  0.0000  -10.0000  20.0000  0.0000  4800.0000
F01: 0.0000  1.0000  0.6667  0.3333  0.0000  80.0000
F02: 0.0000  0.0000  -0.3333  -0.6667  1.0000  40.0000
F03: 0.0000  0.0000  1.3333  -0.3333  0.0000  20.0000

--> Pivote: fila 3, columna 2 (valor 1.3333)
Tabla actual:
Z : 1.0000  0.0000  0.0000  17.5000  0.0000  4950.0000
F01: 0.0000  1.0000  0.0000  0.5000  0.0000  70.0000
F02: 0.0000  0.0000  0.0000  -0.7500  1.0000  45.0000
F03: 0.0000  0.0000  1.0000  -0.2500  0.0000  15.0000

Estado: OPTIMO
Z*   : 4950.0000
x*   : [70.0000, 15.0000]
Slacks: [0.0000, 45.0000, 0.0000]

 Simplex por consola (estilo TORA)
 1) Ingresar modelo y resolver
 2) Demo Kyndryl (Max 60A + 50B)
 3) Salir
Opción: 3
